In [29]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [30]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        for file in glob.glob(directory+"/*.txt"):
            text = readtext(file)
            docs.append((text,directory))
    return docs

data = read_data(['neg','pos'])

In [31]:
import random

random.shuffle(data)
train_data_raw = data[80:]
test_data_raw = data[:80]

In [32]:
import nltk
import re
from collections import Counter

stopwords = nltk.corpus.stopwords.words('english')

def features_from_text(text, POSlist):
    wordcounts = Counter()
    tlen = 0
    
    sentlist = nltk.sent_tokenize(text,language='english')
    for sent in sentlist:
        tokens = nltk.word_tokenize(sent,language='english')
        tokens = [lemma for (lemma,pos) in nltk.pos_tag(tokens) if pos in POSlist]
        wordcounts.update(tokens)
        tlen +=len(tokens)
    return {w:wordcounts[w]/tlen for w in wordcounts}

In [54]:
# poslist = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
# poslist = ['RB', 'RBR', 'RBS']
poslist = ['JJ', 'JJR', 'JJS']


test_data = [(features_from_text(text,poslist),cl) for text,cl in test_data_raw]
train_data = [(features_from_text(text,poslist),cl) for text,cl in train_data_raw]

In [55]:
train_data[13]

({'brian': 0.007874015748031496,
  'palma': 0.023622047244094488,
  'director': 0.007874015748031496,
  'carrie': 0.007874015748031496,
  'mission': 0.007874015748031496,
  'expertise': 0.007874015748031496,
  'story': 0.023622047244094488,
  'characters': 0.031496062992125984,
  'cage': 0.031496062992125984,
  'plays': 0.007874015748031496,
  'santaro': 0.007874015748031496,
  'cop': 0.007874015748031496,
  'boxing': 0.007874015748031496,
  'match': 0.007874015748031496,
  'friend': 0.007874015748031496,
  'commander': 0.007874015748031496,
  'kevin': 0.015748031496062992,
  'dunne': 0.015748031496062992,
  'sinise': 0.023622047244094488,
  'assassination': 0.007874015748031496,
  'place': 0.015748031496062992,
  'tries': 0.007874015748031496,
  'roles': 0.007874015748031496,
  'people': 0.007874015748031496,
  'conspiracy': 0.007874015748031496,
  'film': 0.10236220472440945,
  'flashbacks': 0.023622047244094488,
  'video': 0.007874015748031496,
  'cameras': 0.007874015748031496,
  '

In [56]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

In [57]:
from sklearn import linear_model, datasets

allfeatures = [w for w in docfreq if docfreq[w] >= 5]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

In [58]:
train_vec[86]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.043478260869565216,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.004347826086956522,
 0.004347826086956522,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.0,
 0.004347826086956522,
 0.004347826086956522,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.008695652173913044,
 0.0,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.004347826086956522,
 0.004347826086956522,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.017391304347826087,
 0.0,
 0.0,
 0.0,
 0.008695652173913044,
 0.0,
 0.0,
 0.0,
 0.017391304347826087,
 0.0,
 0.0,
 0.0,
 0.004347826086956522,
 0.0,
 0.0,
 0.0,
 0.0,


In [59]:
train_label[86]

'neg'

In [60]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True)
logreg.fit(train_vec,train_label)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/byigit/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s finished


LogisticRegression(C=1000000000.0, verbose=True)

In [61]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

In [62]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

81.2 Prozent korrekt
